In [ ]:
import cv2
import numpy as np
import serial
import time


# Load YOLOv4-tiny model
model = cv2.dnn.readNetFromDarknet('yolov4-tiny.cfg', 'yolov4-tiny.weights')
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Get the output layer names
layer_names = model.getLayerNames()
output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]

# Load the class names from the coco.names file
with open('coco.names.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Open the webcam
#droidcam_ip = 'http://172.20.10.2:4747/video'
# Connect to Droidcam
#cap = cv2.VideoCapture(droidcam_ip)
cap = cv2.VideoCapture(0)


# Set up serial communication
ser = serial.Serial("COM4", 9600)


def sendCommand(command):
    ser.write(command.encode('ascii'))
    time.sleep(0.1)
    
for i in range(0,10):
    sendCommand('X')

time.sleep(2)

while True:
    circle_detected = False
    # Read the current frame from the webcam
    ret2, frame2 = cap.read()
    
    # If the frame was not successfully read, then we have reached the end of the video
    if not ret2:
        break
    
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    # Define the lower and upper boundaries for the red color
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create a mask for the red color
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Apply a series of morphological operations to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours of the red circles
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour that is approximately circular
    largest_circle = None
    max_radius = 0
    
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        
        if circularity > 0.8:  # Adjust this threshold to set the desired circularity
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius > max_radius:
                max_radius = radius
                largest_circle = (x, y, radius)
    
    if largest_circle is not None:
        (x, y, radius) = largest_circle
        center = (int(x), int(y))
        radius = int(radius)
        
        # Draw the circle on the frame
        cv2.circle(frame2, center, radius, (0, 255, 0), 2)
        
        # Get the center of the frame
        frame_center = (frame2.shape[1] // 2, frame2.shape[0] // 2)
        
        # Set the proximity threshold as a fraction of the frame width
        proximity_threshold = 0.2  # Adjust this value to set the desired threshold
        
        # Calculate the maximum allowed distance from the center
        max_distance = proximity_threshold * frame2.shape[1]
        
        # Calculate the actual distance between the circle center and the frame center
        distance = np.sqrt((center[0] - frame_center[0]) ** 2 + (center[1] - frame_center[1]) ** 2)
        
        # Check if the distance is within the maximum allowed distance
        if distance < max_distance:
            circle_detected = True
        else:
            print("Circle is not close to the middle")
#-----------------------------------------------------------------------------------------------------------------------------
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Resize the frame to a size divisible by the network's stride
    net_input_size = (416, 416)  # the network's input size
    stride = 32  # the network's stride
    height, width, _ = frame.shape
    new_height = (height // stride) * stride
    new_width = (width // stride) * stride
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Preprocess the input frame
    blob = cv2.dnn.blobFromImage(resized_frame, 1 / 255.0, net_input_size, swapRB=True, crop=False)

    # Set the input to the network and run a forward pass
    model.setInput(blob)
    layer_outputs = model.forward(output_layers)

    # Postprocess the outputs
    boxes = []
    confidences = []
    class_ids = []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Draw the bounding boxes and labels
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    person_detected = False
    person_x = None # The x coordinate of the person's center
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            color = colors[i]
            class_id = class_ids[i]
            label = str(classes[class_id])
            if class_id == 0:  # Person is detected
                person_detected = True
                person_x = x + w / 2 # Calculate the person's center x coordinate
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 5), font, 1, color, 2)

    #Show the output
    cv2.imshow("Frame", frame2)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    # Send command to Arduino based on person detection
        # Send command to Arduino based on person detection
    if circle_detected:
        sendCommand('F')  # forwrd
        ser.reset_input_buffer()
        if person_detected:
            # Compare the person's center x coordinate to the frame's center x coordinate
            frame_center_x = width / 2
            if person_x < frame_center_x: # Person is on the left side of the frame
                sendCommand('R')  # Turn Right
                ser.reset_input_buffer() 
            elif person_x > frame_center_x: # Person is on the right side of the frame
                sendCommand('L')  # Turn Left
                ser.reset_input_buffer()
            else: # Person is on the center of the frame
                sendCommand("M") # Stop
    else:
        sendCommand("M")
    # if circle_detected:
    #     sendCommand('F')  # forwrd
    #     ser.reset_input_buffer()


# Release the resources
cap.release()
cv2.destroyAllWindows()
ser.close()